In [1]:
import torch
import torch.nn.functional as F
from utils import load_our_sae, load_concatenated_sae, import_connor_sae

In [2]:
# load a single SAE
hook_point_head_index = 4
model, our_single_attn_head_sae, activations_loader = load_our_sae(hook_point_head_index=hook_point_head_index)

Loading SAE for head # 4
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [3]:
model, our_concatenated_sae, activations_loader = load_concatenated_sae()
connor_sae = import_connor_sae()

Loading SAE for head # 0
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Loading SAE for head # 1
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

...

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Loading SAE for head # 7
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Concatenating all SAEs from individual attention heads


## Getting feature activations

In [12]:
# get model cache
batch_tokens = activations_loader.get_batch_tokens(batch_size=1)
B, T = batch_tokens.shape
_, cache = model.run_with_cache(batch_tokens, prepend_bos=True)

In [4]:
## forward pass through SAEs
# forward pass outputs (sae_out, feature_acts, loss, mse_loss, l1_loss, ghost_grad_loss)
single_attn_head_feature_acts = our_single_attn_head_sae(
    cache[our_single_attn_head_sae.cfg.hook_point][:, :, hook_point_head_index, :]
)[2]

In [7]:
# Connor's SAE
hook_point = connor_sae.cfg.hook_point
connor_feature_acts = connor_sae(
    cache[hook_point].view(B, T, -1)
)[2]

In [8]:
# our concatenated SAE
hook_point = our_concatenated_sae.cfg.hook_point
our_concatenated_feature_acts = our_concatenated_sae(
    cache[hook_point].view(B, T, -1)
)[2]

### Checking the rank of our concatenated SAE

In [5]:
torch.linalg.matrix_rank(our_concatenated_sae.W_dec)

tensor(512)

In [10]:
torch.linalg.matrix_rank(our_concatenated_sae.W_dec @ model.W_O[1].flatten(start_dim=0, end_dim=1))

tensor(504)

In [11]:
torch.linalg.matrix_rank(connor_sae.W_dec @ model.W_O[1].flatten(start_dim=0, end_dim=1))

tensor(496)

In [17]:
cache.keys()

dict_keys(['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'ln_final.hook_scale', 'ln_final.

In [16]:
torch.linalg.matrix_rank(cache['blocks.1.attn.hook_z'].flatten(start_dim=-2, end_dim=-1))

tensor([512])

In [20]:
torch.linalg.matrix_rank(cache['blocks.1.hook_attn_out'])

tensor([497])